In [2]:
%reset
from sympy import *
import numpy as np

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
#paramaters
m, g, t, l = symbols("m, g, t, l")


G = Matrix([0,0,g])

In [4]:
ixx, ixy, ixz, iyx, iyy, iyz, izx, izy, izz = symbols("ixx, ixy, ixz, iyx, iyy, iyz, izx, izy, izz")

MI = Matrix([[ixx,   0,   0],
             [  0, iyy,   0],
             [  0,   0, izz]])

In [5]:
#Interial Frame Vectors
x = Function('x')(t)
y = Function('y')(t)
z = Function('z')(t)

xd     = diff(x,t)
yd     = diff(y,t)
zd     = diff(z,t)

xdd    = diff(x,(t,2))
ydd    = diff(y,(t,2))
zdd    = diff(z,(t,2))

X_i    = Matrix([x,y,z])
Xd_i   = Matrix([xd, yd, zd])
Xdd_i  = Matrix([xdd, ydd, zdd])



In [6]:
#Rotation Setup
#Euler
phi    = Function('phi')(t)
theta  = Function('theta')(t)
psi    = Function('psi')(t)

T_i    = Matrix([phi, theta, psi])

#Quaternians
q0     = Function('q0')(t)
q1     = Function('q1')(t)
q2     = Function('q2')(t)
q3     = Function('q3')(t)

q = Matrix([q0,q1,q2,q3])

phi   = atan2(2*(q0*q1 + q2*q3), q0**2 - q1**2 - q2**2 + q3**2)
theta = asin(2*(q0*q2 - q3*q1))
psi   = atan2(2*(q0*q3 + q2*q1), q0**2 + q1**2 - q2**2 - q3**2)

wx    = Function('wx')(t)
wy    = Function('wy')(t)
wz    = Function('wz')(t)

wxd   = diff(wx,t)
wyd   = diff(wy,t)
wzd   = diff(wz,t)

w     = Matrix([wx,wy,wz])

wd    = Matrix([wxd,wyd,wzd])

p    = Matrix([0,wx,wy,wz])

Q     = Matrix([[q0,-q1,-q2,-q3],
                [q1, q0, q3,-q2],
                [q2,-q3, q0, q1],
                [q3, q2,-q1, q0]])

qd = (-1/2) * Matrix([p[0]*q0 - p[1]*q1 - p[2]*q2 - p[3]*q3,
                      p[0]*q1 + p[1]*q0 + p[2]*q3 - p[3]*q2,
                      p[0]*q2 - p[1]*q3 + p[2]*q0 + p[3]*q1,
                      p[0]*q3 + p[1]*q2 - p[2]*q1 + p[3]*q0])



In [7]:
R = Matrix([[1-2*q2**2-2*q3**2      ,         2*q1*q2-2*q0*q3,         2*q1*q3+2*q0*q2],
            [        2*q1*q2+2*q0*q3, 1-2*q1**2-2*q3**2      ,         2*q2*q3-2*q0*q1],
            [        2*q1*q3-2*q0*q2,         2*q2*q3+2*q0*q1, 1-2*q1**2-2*q2**2      ]])
R

Matrix([
[  -2*q2(t)**2 - 2*q3(t)**2 + 1, -2*q0(t)*q3(t) + 2*q1(t)*q2(t),  2*q0(t)*q2(t) + 2*q1(t)*q3(t)],
[ 2*q0(t)*q3(t) + 2*q1(t)*q2(t),   -2*q1(t)**2 - 2*q3(t)**2 + 1, -2*q0(t)*q1(t) + 2*q2(t)*q3(t)],
[-2*q0(t)*q2(t) + 2*q1(t)*q3(t),  2*q0(t)*q1(t) + 2*q2(t)*q3(t),   -2*q1(t)**2 - 2*q2(t)**2 + 1]])

In [8]:
#Newton & Quaternians

F1, F2, F3, F4 = symbols("F1, F2, F3, F4")
T1, T2, T3, T4 = symbols("T1, T2, T3, T4")

F1_rotor = Matrix([0,0,F1]) 
F2_rotor = Matrix([0,0,F2])
F3_rotor = Matrix([0,0,F3])
F4_rotor = Matrix([0,0,F4])

T1_rotor = Matrix([0,0,T1])
T2_rotor = Matrix([0,0,T2])
T3_rotor = Matrix([0,0,T3])
T4_rotor = Matrix([0,0,T4])

R1_rotor = Matrix([l,0,0])
R2_rotor = Matrix([0,l,0])
R3_rotor = Matrix([-l,0,0])
R4_rotor = Matrix([0,-l,0])



In [9]:
XDD = ((m*G + R @ F1_rotor + R @ F2_rotor + R @ F3_rotor + R @ F4_rotor)/m)

In [10]:
Torque =  (T1_rotor + T2_rotor + T3_rotor + T4_rotor + R1_rotor.cross(F1_rotor) + R2_rotor.cross(F2_rotor) + R3_rotor.cross(F3_rotor) + R4_rotor.cross(F4_rotor)) 
wd = MI.inv() @ Torque - MI.inv() @ (w.cross(MI @ w))

In [11]:
#xdd
XDD[0].simplify()

2*(q0(t)*q2(t) + q1(t)*q3(t))*(F1 + F2 + F3 + F4)/m

In [12]:
#ydd
XDD[1].simplify()

2*(q0(t)*q1(t) - q2(t)*q3(t))*(-F1 - F2 - F3 - F4)/m

In [13]:
#zdd
XDD[2].simplify()

(-F1*(2*q1(t)**2 + 2*q2(t)**2 - 1) - F2*(2*q1(t)**2 + 2*q2(t)**2 - 1) - F3*(2*q1(t)**2 + 2*q2(t)**2 - 1) - F4*(2*q1(t)**2 + 2*q2(t)**2 - 1) + g*m)/m

In [14]:
#wxd
wd[0]

(F2*l - F4*l)/ixx - (-iyy*wy(t)*wz(t) + izz*wy(t)*wz(t))/ixx

In [15]:
#wyd
wd[1]

(-F1*l + F3*l)/iyy - (ixx*wx(t)*wz(t) - izz*wx(t)*wz(t))/iyy

In [16]:
#wzd
wd[2]

-(-ixx*wx(t)*wy(t) + iyy*wx(t)*wy(t))/izz + (T1 + T2 + T3 + T4)/izz

In [17]:
wd.subs(q0,1).subs(q1,0).subs(q2,0).subs(q3,0).subs(F1,1.4).subs(F2,1.4).subs(F3,1.4).subs(F4,1.4).subs(T1,0).subs(T2,0).subs(T3,0).subs(T4,0).subs(wx,0).subs(wy,0).subs(wz,.125)

Matrix([
[0],
[0],
[0]])

In [18]:
wd.subs(q0,.7).subs(q1,0).subs(q2,0).subs(q3,-0.7).subs(F1,1.2).subs(F2,1.4).subs(F3,1.6).subs(F4,1.4).subs(T1,0).subs(T2,0).subs(T3,0).subs(T4,0).subs(wx,0).subs(wy,0).subs(wz,0)

Matrix([
[        0],
[0.4*l/iyy],
[        0]])

In [19]:
XDD.subs(q0,0.715421374563456).subs(q1,-0.0881583494193193).subs(q2,0.0881583494193194).subs(q3,0.701589698775332).subs(F1,1.4).subs(F2,1.4).subs(F3,1.4).subs(F4,1.4)

Matrix([
[      0.0136570303204636/m],
[        1.39911920214579/m],
[(g*m + 5.42590956157961)/m]])

In [22]:
XDD.subs(q0,1).subs(q1,0.0353284606605302).subs(q2,-0.0358083672097288).subs(q3,0.705833241123056).subs(F1,1.4).subs(F2,1.4).subs(F3,1.4).subs(F4,1.4)

Matrix([
[      -0.121770491559566/m],
[      -0.678755801331974/m],
[(g*m + 5.57166019989746)/m]])

In [24]:
wd.subs(q0,0.715421374563456).subs(q1,0).subs(q2,0).subs(q3,0.715421374563456).subs(F1,1.2).subs(F2,1.4).subs(F3,1.6).subs(F4,1.4).subs(T1,0.0001).subs(T2,0.0001).subs(T3,0.0001).subs(T4,0.0001).subs(wx,0).subs(wy,0).subs(wz,0)

Matrix([
[         0],
[ 0.4*l/iyy],
[0.0004/izz]])